# Dataset API (TensorFlow 1.4)

# 1. Dataset

tf.data.Dataset là API chuẩn của TensorFlow, hỗ trợ việc đọc và tiền xử lý dữ liệu (input pipeline). API này không sử dụng cơ chế Queue tổng quát như trước mà được implement bằng C, optimize cho việc đọc dữ liệu nên có perfomance cao hơn.

Lợi ích của Dataset API:
- Có các hàm high-level (batching, prefetching, ...), input pipeline code ngắn đi rất nhiều so với cơ chế Queue.
- Hỗ trợ tạo data từ nhiều nguồn: TFRecord, từ Numpy array, từ Tensor hằng số, etc.
- Hỗ trợ `Estimator` API: Dataset được tạo và nằm trong `input_fn`.
- Được thiết kế tổng quát, có thể áp dụng cho nhiều loại dữ liệu: hình ảnh, text, âm thanh, etc.
- Tiền xử lý có thể được chia làm nhiều hàm nhỏ: khả năng reuse cao.

API này bao gồm 2 thành phần chính:
- tf.data.Dataset: object chứa tập dữ liệu.
- tf.data.Iterator: object dùng để duyệt và lấy dữ liệu từ dataset.

`tf.data.Dataset`: chứa một chuỗi các element, mỗi element là một bộ Tensor (được gọi là component). 
Vd, trong bài toán image classification thì 1 element gồm 2 Tensor (image, label).

Các element trong Dataset cần có cấu trúc tương tự nhau: *tf.DType* và *tf.TensorShape* của các component tương ứng của element phải giống nhau.
Để coi cấu trúc của Dataset, ta dùng:
- Dataset.output_types: coi type của các component của 1 element.
- Dataset.output_shapes: coi shape của các component của 1 element.

In [ ]:
# Inspect type and shape of components
# Modified from: https://www.tensorflow.org/programmers_guide/datasets

# Each element consists of only 1 Tensor
dataset1 = tf.data.Dataset.from_tensor_slices(tf.random_uniform([4, 10]))
print(dataset1.output_types)  # ==> "tf.float32"
print(dataset1.output_shapes)  # ==> "(10,)"

# 2 Tensors
dataset2 = tf.data.Dataset.from_tensor_slices(
   (tf.random_uniform([4]),
    tf.random_uniform([4, 100], maxval=100, dtype=tf.int32)))
print(dataset2.output_types)  # ==> "(tf.float32, tf.int32)"
print(dataset2.output_shapes)  # ==> "((), (100,))"

# Each element is a tuple of Tensors
dataset3 = tf.data.Dataset.zip((dataset1, dataset2))
print(dataset3.output_types)  # ==> (tf.float32, (tf.float32, tf.int32))
print(dataset3.output_shapes)  # ==> "(10, ((), (100,)))"

Dataset được tạo bằng 2 cách:
1. Tạo từ Tensor: tạo Dataset từ 1 list các Tensor hoặc đọc từ TFRecord.
2. Tạo từ Dataset: `transform` Dataset, thường dùng để preprocess.

# 2. Iterator

Để lấy element từ Dataset đã tạo, ta sử dụng `tf.data.Iterator`.
Có 4 loại Iterator, từ đơn giản đến phức tạp:
1. `one-shot`: duyệt qua dữ liệu theo thứ tự, Dataset phải được tạo trước, không thể truyền tham số vào để thay đổi cách duyệt được.
2. `initializable`: ta phải khởi tạo Iterator trước khi lấy element ra (có thể truyền tham số vào *feed_dict* để cung cấp giá trị cho `place_holder` mà Dataset dùng). Lưu ý: chỉ truyền được 1 lần khi khởi tạo Iterator nên *place_holder* này không thể lưu data (vd có thể dùng để lưu tên file mà Dataset sẽ đọc).
3. `reinitializable`: iterator này có thể được reinit nhiều lần, mỗi lần đọc từ 1 Dataset khác nhau (dùng cùng 1 iterator, build ra graph model và chạy trên tập train/val).
4. `feedable`: một kiểu meta-iterator, giúp chọn iterator muốn dùng (mỗi iterator là của 1 Dataset khác nhau) mà không cần reinit lại iterator.

**Lưu ý: v1.4 thì TF khuyến cáo chỉ nên dùng `one-shot` iterator với API Estimator.**

### 1. One-shot iterator

In [ ]:
# One-shot iterator example
# Modified from: https://www.tensorflow.org/programmers_guide/datasets

dataset = tf.data.Dataset.range(100)
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

for i in range(100):
    value = sess.run(next_element)
    assert i == value

### 2. Initializable iterator

In [ ]:
# Initializable iterator example
# Modified from: https://www.tensorflow.org/programmers_guide/datasets

max_value = tf.placeholder(tf.int64, shape=[])
dataset = tf.data.Dataset.range(max_value)
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

# Initialize an iterator over a dataset with 10 elements.
sess.run(iterator.initializer, feed_dict={max_value: 10})
for i in range(10):
    value = sess.run(next_element)
    assert i == value

# Initialize the same iterator over a dataset with 100 elements.
sess.run(iterator.initializer, feed_dict={max_value: 100})
for i in range(100):
    value = sess.run(next_element)
    assert i == value

### 3. Reinitializable iterator

In [ ]:
# Reinitializable iterator example
# Modified from: https://www.tensorflow.org/programmers_guide/datasets

# Define training and validation datasets with the same structure.
training_dataset = tf.data.Dataset.range(100).map(
    lambda x: x + tf.random_uniform([], -10, 10, tf.int64))
validation_dataset = tf.data.Dataset.range(50)

# A reinitializable iterator is defined by its structure. We could use the
# `output_types` and `output_shapes` properties of either `training_dataset`
# or `validation_dataset` here, because they are compatible.
iterator = Iterator.from_structure(training_dataset.output_types,
                                   training_dataset.output_shapes)
next_element = iterator.get_next()

training_init_op = iterator.make_initializer(training_dataset)
validation_init_op = iterator.make_initializer(validation_dataset)

# Run 20 epochs in which the training dataset is traversed, followed by the
# validation dataset.
for _ in range(20):
    # Initialize an iterator over the training dataset.
    sess.run(training_init_op)
    for _ in range(100):
        sess.run(next_element)

    # Initialize an iterator over the validation dataset.
    sess.run(validation_init_op)
        for _ in range(50):
    sess.run(next_element)

### 4. Feedable iterator

In [ ]:
# Feedable iterator example
# Modified from: https://www.tensorflow.org/programmers_guide/datasets

# Define training and validation datasets with the same structure.
training_dataset = tf.data.Dataset.range(100).map(
    lambda x: x + tf.random_uniform([], -10, 10, tf.int64)).repeat()
validation_dataset = tf.data.Dataset.range(50)

# A feedable iterator is defined by a handle placeholder and its structure. We
# could use the `output_types` and `output_shapes` properties of either
# `training_dataset` or `validation_dataset` here, because they have
# identical structure.
handle = tf.placeholder(tf.string, shape=[])
iterator = tf.data.Iterator.from_string_handle(
    handle, training_dataset.output_types, training_dataset.output_shapes)
next_element = iterator.get_next()

# You can use feedable iterators with a variety of different kinds of iterator
# (such as one-shot and initializable iterators).
training_iterator = training_dataset.make_one_shot_iterator()
validation_iterator = validation_dataset.make_initializable_iterator()

# The `Iterator.string_handle()` method returns a tensor that can be evaluated
# and used to feed the `handle` placeholder.
training_handle = sess.run(training_iterator.string_handle())
validation_handle = sess.run(validation_iterator.string_handle())

# Loop forever, alternating between training and validation.
while True:
    # Run 200 steps using the training dataset. Note that the training dataset is
    # infinite, and we resume from where we left off in the previous `while` loop
    # iteration.
    for _ in range(200):
        sess.run(next_element, feed_dict={handle: training_handle})

    # Run one pass over the validation dataset.
    sess.run(validation_iterator.initializer)
    for _ in range(50):
        sess.run(next_element, feed_dict={handle: validation_handle})

# 3. Use case

### Full pipeline với Estimator API

Để dùng Dataset API với Estimator API, ta implement `input_fn` bằng cách dùng Dataset, iterator và return lại Tensor lấy từ Dataset.

Ví dụ để tạo input pipeline đọc dữ liệu từ TFRecord cho bài toán image classification:
1. `input_fn` nhận file_patterns và lấy hết tất cả file TFRecord có pattern này.
2. Implement hàm `parser` để parse proto buff message với input là 1 record khi đọc được từ TFRecord. Hàm này return 2 Tensor là image đã decode và label.
3. Implement hàm `preprocess`, nhận 2 Tensor của hàm trên và build graph để tiền xử lý ảnh. Hàm này return theo định dạng của `input_fn` mà Estimator yêu cầu là features là 1 dict và labels là 1 Tensor.
4. Tạo `tf.data.TFRecordDataset` từ list tên file TFRecord. Dataset này sẽ đọc từ TFRecord và trả về string là protocol buffer message được lưu trong đó (cơ chế tự advance phần tử tiếp theo, advance qua file tiếp theo hay buffer các phần tử đã có sẵn, không cần tự cài đặt).
5. Gọi hàm `parser` để parse record từ Dataset trên bằng cách dùng hàm `map` (apply 1 hàm cho từng phần tử).
6. Gọi hàm `preprocess` nhận 2 Tensor từ hàm trên và tiền xử lý dữ liệu cũng bằng hàm `map`.
7. Thực hiện shuffle.
8. Repeat để lặp train vô hạn (thêm tham số là số lần lặp để hữu hạn). Ta sẽ truyền num_steps vào Estimator để dừng việc train/eval.
9. Gọi prefetch để tạo nhiều thread đọc dữ liệu.
10. Tạo one-shot iterator và return kết quả.

In [ ]:
def input_fn(file_patterns, is_training, batch_size):
    """Input function for Resnet Estimator."""
    filenames = []
    for pattern in file_patterns.split(","):
        filenames.extend(tf.gfile.Glob(pattern))

    def parser(record):
        """Parse tf.Example protobuf."""
        keys_to_features = {
            "image/data": tf.FixedLenFeature((), tf.string),
            "label": tf.FixedLenFeature((), tf.int64)
        }
        parsed = tf.parse_single_example(record, features=keys_to_features)

        image = tf.image.decode_image(parsed["image/data"], channels=3)
        label = parsed["label"]

        return image, label

    def preprocess(image, label):
        """Preprocess image for Resnet."""
        image_size = resnet_v1.resnet_v1_101.default_image_size
        image = vgg_preprocessing.preprocess_image(image, image_size,
                                                   image_size,
                                                   is_training=is_training)
        label = tf.to_int32(label)
        return {"image": image}, label

    with tf.device('/cpu:0'):
        # Create the dataset
        dataset = tf.data.TFRecordDataset(filenames, buffer_size=256 * 2 ** 20)
        dataset = dataset.map(parser, num_parallel_calls=batch_size * 10)
        dataset = dataset.map(preprocess,
                              num_parallel_calls=batch_size * 10)
        dataset = dataset.shuffle(buffer_size=1000)
        dataset = dataset.repeat()
        dataset = dataset.batch(batch_size=batch_size)
        dataset = dataset.prefetch(batch_size)

        # Create iterator for the dataset
        iterator = dataset.make_one_shot_iterator()
        features, labels = iterator.get_next()

    return features, labels

### Đọc từng dòng trong file text

Dataset API hỗ trợ đọc dữ liệu dạng text hoặc CSV (mỗi dòng là 1 element).

Ví dụ để tạo input pipeline đọc từ file text:
1. Tạo dataset lưu tên file.
2. Tạo TextLineDataset từ từng file của dataset trên.
3. Skip dòng đầu.
4. Filter những dòng comment.
5. Flatten các element của tất cả file (cho cùng cấp).

In [ ]:
# Read line-by-line from text file example
# Modified from: https://www.tensorflow.org/programmers_guide/datasets

filenames = ["/var/data/file1.txt", "/var/data/file2.txt"]

dataset = tf.data.Dataset.from_tensor_slices(filenames)

# Use `Dataset.flat_map()` to transform each file as a separate nested dataset,
# and then concatenate their contents sequentially into a single "flat" dataset.
# * Skip the first line (header row).
# * Filter out lines beginning with "#" (comments).
dataset = dataset.flat_map(
    lambda filename: (
        tf.data.TextLineDataset(filename)
        .skip(1)
        .filter(lambda line: tf.not_equal(tf.substr(line, 0, 1), "#"))))
